# Introduction

In this tutorial we demonstrate the use of a light-weight `CrystalMap` class to store and manipulate orientations, crystal phases and other properties associated with every spatial coordinate in a 1D or 2D space.

The `CrystalMap` class is inspired by MTEX' `EBSD` class. It is developed to interface more easily with the scientific Python stack.

Orientations and other properties acquired from a super-duplex stainless steel EBSD data set with two phases, austenite and ferrite, are used as example data. The data is available here: http://folk.ntnu.no/hakonwii/files/orix-demos/, courtesy of Prof. Jarle Hjelen (Norwegian University of Science and Technology, Norway).

This functionaility has been checked to run in orix-0.3.0 (July 2020). Bugs are always possible, do not trust the code blindly, and if you experience any issues please report them here: https://github.com/pyxem/orix-demos/issues. Suggestions for improvement of the functionality we cover in this tutorial are also welcome.

# Contents

1. <a href='#obtain-crystalmap'> Import/create and save a CrystalMap object</a>
2. <a href='#inspect-phases'> Inspect and manipulate phases</a>
3. <a href='#inspect-data'> Inspect orientation data</a>
4. <a href='#inspect-properties'> Inspect, add and delete map properties</a>
5. <a href='#select-data'> Select and manipulate data based upon criteria</a>
6. <a href='#plot-maps'> Plot maps</a>

Import orix classes and various dependencies

In [ ]:
%matplotlib qt5

from diffpy.structure import Atom, Lattice, Structure
import matplotlib.pyplot as plt
import numpy as np

from orix.io import load, save
from orix.quaternion.rotation import Rotation
from orix.quaternion.orientation import Orientation
from orix.crystal_map import CrystalMap, PhaseList, Phase
from orix import plot

# <a id='obtain-crystalmap'></a> 1. Import/create and save a CrystalMap object

A `CrystalMap` object can be obtained either by reading an orientation data set
stored in a format supported by `orix` using the `load` function, or by passing
the necessary arrays to the `CrystalMap.__init__()` method. Two formats are
supported, in addition to `orix`'s own HDF5 format: Data in the ang format
produced by the softwares EDAX TSL OIM Data Collection v7, NanoMegas Astar
Index, and EMsoft v4/v5 via the `EMdpmerge` program, and data in EMsoft v4/v5
HDF5 files produced by the `EMEBSDDI` program.

Let's get a crystal map from an .ang file produced by EMsoft

In [ ]:
datadir = '/home/hakon/phd/data/jarle_emsoft/sdss/em/'
fname = 'sdss_ferrite_austenite.ang'

cm = load(datadir + fname)

# Let's print a nice, informative representation of the data
cm

Note that the name and symmetry of the phases present in the data were obtained from the .ang file header. The indexing properties returned by EMsoft in their .ang files are the pattern image quality (iq) (according to Niels Krieger Lassen's method), and the highest normalized dot product (dp) between the experimental and best matching simulated pattern.

We can obtain the same `CrystalMap` object by reading each array from the .ang files ourselves and passing this to `CrystalMap.__init__()`

In [ ]:
# Read each column from the file
euler1, euler2, euler3, x, y, iq, dp, phase_id = np.loadtxt(
    datadir + fname, unpack=True)

# Create a Rotation object from Euler angles
euler_angles = np.column_stack((euler1, euler2, euler3))
rotations = Rotation.from_euler(euler_angles)

# Create a property dictionary
properties = {"iq": iq, "dp": dp}

# Create unit cells of the phases
structures = [
    Structure(
        title="austenite",
        atoms=[Atom("fe", [0] * 3)],
        lattice=Lattice(0.360, 0.360, 0.360, 90, 90, 90)
    ),
    Structure(
        title="ferrite",
        atoms=[Atom("fe", [0] * 3)],
        lattice=Lattice(0.287, 0.287, 0.287, 90, 90, 90)
    ),
]
phase_list = PhaseList(
    names=["austenite", "ferrite"],
    symmetries=["432", "432"],
    structures=structures,
)

# Create a CrystalMap object
cm2 = CrystalMap(
    rotations=rotations,
    phase_id=phase_id,
    x=x,
    y=y,
    phase_list=phase_list,
    prop=properties,
)
cm2.scan_unit = "um"

cm2

The only supported format to write a `CrystalMap` object to is `orix`' own
HDF5 format

In [ ]:
save(
    filename=datadir + "sdss_ferrite_austenite2.h5",
    object2write=cm,
    overwrite=False,  # Default
)

Read the file contents back into a `CrystalMap` object using `orix`' `load`
function.

All contents in this file can be inspected using any HDF5 viewer and read back
into Python using the `h5py` library (which we use). 

# <a id='inspect-phases'></a> 2. Inspect and manipulate phases

The phases are stored in a `PhaseList` object in the `CrystalMap.phases` attribute

In [ ]:
cm.phases

This list can be indexed by phase ID or name

In [ ]:
cm.phases[1]

In [ ]:
cm.phases["austenite"]

In [ ]:
cm.phases[1:]

In [ ]:
cm.phases["austenite", "ferrite"]

When asking for a single phase, either by an integer or a single string, a `Phase` object was returned. In the other cases, a `PhaseList` object was returned

In [ ]:
print(
    type(cm.phases[1]),
    type(cm.phases[1:])
)

The phase name, symmetry, color and structure can be accessed for the full phase list or a single phase

In [ ]:
print(cm.phases.names)
print([symmetry.name for symmetry in cm.phases.symmetries])
print(cm.phases.colors)
print(cm.phases.structures)

Note that the structures' representations are empty lists since no atoms have been added to them yet.

In [ ]:
cm.phases["austenite"]
print(cm.phases["austenite"].name)
print(cm.phases["austenite"].symmetry.name)
print(cm.phases["austenite"].color)
print(cm.phases["austenite"].structure)

These attributes (not the phase ID) can be set *per phase*

In [ ]:
cm.phases["austenite"].name = "Austenite"

cm.phases["Austenite"].structure = Structure(
    lattice=Lattice(0.36, 0.36, 0.36, 90, 90, 90))
print(cm.phases["Austenite"].structure)

cm.phases["Austenite"].color = "lime"  # Yields RGB tuple (0, 1, 0)
print(cm.phases["Austenite"].color_rgb)

cm.phases

Valid color strings can be found here: https://matplotlib.org/3.1.0/tutorials/colors/colors.html

Valid point group names to use when setting the phase symmetry are

In [ ]:
from orix.quaternion.symmetry import _groups

[point_group.name for point_group in _groups]

In [ ]:
cm.phases["Austenite"].symmetry = "m-3m"

cm.phases

Let's revert to the correct point group (and the name, for convenience)

In [ ]:
cm.phases["Austenite"].name = "austenite"
cm.phases["austenite"].symmetry = "432"

cm.phases

We can add a phase by giving its name and point group symmetry

In [ ]:
cm.phases["sigma"] = "4/mmm"

cm.phases

When adding a phase to the phase list like this, the phases' structure contains no atoms and the default lattice parameters are used

In [ ]:
cm.phases["sigma"].structure.lattice.abcABG()

So let's set this

In [ ]:
cm.phases["sigma"].structure.lattice = Lattice(0.880, 0.880, 0.880, 90, 90, 90)
print(cm.phases["sigma"].structure.lattice)

If some data points are considered as not indexed, a "not_indexed" phase can be added to the phase list to keep track of these points

In [ ]:
cm.phases.add_not_indexed()

cm.phases

No points in this data set are considered not indexed. A phase list with only the phases in the data, is stored in the `CrystalMap.phases_in_data` attribute

In [ ]:
cm.phases_in_data

We can of course remove a phase from the phase list, either by its name or phase ID

In [ ]:
del cm.phases["sigma"]
del cm.phases[-1]

cm.phases

We can create a phase list by calling `PhaseList.__init__()`

In [ ]:
PhaseList(
    names=['al', 'cu'],
    symmetries=['m-3m', 'm3m'],  # Note that m3m = m-3m
    colors=['lime', 'xkcd:violet'],
    ids=[0, 1],
    structures=[
        Structure(
            atoms=[Atom("al", [0] * 3)],
            lattice=Lattice(0.405, 0.405, 0.405, 90, 90, 90)
        ),
        Structure(
            atoms=[Atom("cu", [0] * 3)],
            lattice=Lattice(0.361, 0.361, 0.361, 90, 90, 90)
        )
    ]
)

or by creating `Phase` objects and passing these to the first argument in `PhaseList.__init__()` as a list (or single `Phase` objects)

In [ ]:
al = Phase(name='al', symmetry='m-3m', color="C0")
cu = Phase(
    color="C1",
    structure=Structure(
        title="cu",
        lattice=Lattice(0.361, 0.361, 0.361, 90, 90, 90)
    )
)

PhaseList([al, cu])

Note that the Cu phase name was retrieved from the `Structure` object.

If we want a shallow copy of the phase list

In [ ]:
pl = cm.phases
pl["ferrite"].color = "red"

cm.phases

If we want a deep copy of the phase list

In [ ]:
pl = cm.phases.deepcopy()
pl["chi"] = "-43m"
print(pl, "\n")

print(cm.phases)

# <a id='inspect-data'></a> 3. Inspect orientation data

Orientations are stored as rotations in a `Rotation` object

In [ ]:
cm.rotations

Orientations *per phase* can be obtained by applying the phase symmetry

In [ ]:
o_austenite = cm["austenite"].orientations

o_austenite

The above is equivalent to

In [ ]:
Orientation(cm["austenite"].rotations).set_symmetry(
    cm["austenite"].phases[1].symmetry)

Orientation angles and axes are readily available

In [ ]:
o_austenite.angle

In [ ]:
# Obtain as a numpy.ndarray
o_austenite.angle.data

In [ ]:
o_austenite.axis.data

# <a id='inspect-properties'></a> 4. Inspect, add and delete map properties

Map properties are stored in the `CrystalMap.prop` attribute dictionary

In [ ]:
cm.prop

All properties in this dictionary are also available directly from the `CrystalMap` as attributes

In [ ]:
cm.iq

In [ ]:
cm.dp

We can add a map property by specifying its name and an initial value in each map point

In [ ]:
cm.prop["grain_boundary"] = 0

cm.grain_boundary

In [ ]:
cm.prop["grain_boundary2"] = np.arange(cm.size, dtype=int)

cm.grain_boundary2

We can also delete a property from the `prop` dictionary

In [ ]:
del cm.prop["grain_boundary2"]

cm.prop

# <a id='select-data'></a> 5. Select and manipulate data based upon criteria

We can select data in a crystal map in three ways:
1. by phase name or "indexed"/"not_indexed"
2. by a slice
3. by a boolean array

Getting all data belonging to one phase

In [ ]:
cm["austenite"]

or two phases

In [ ]:
cm["austenite", "ferrite"]

or all indexed points

In [ ]:
cm["indexed"]

or all non-indexed points

In [ ]:
cm["not_indexed"]

When slicing a crystal map, it is important to know the data size and shape

In [ ]:
cm.size

In [ ]:
cm.shape

So, to get the data within a rectangle

In [ ]:
cm[20:50, 40:90]

The most powerful way to select data is by requiring a certain criteria

In [ ]:
dp_mean = cm.dp.mean()
print(dp_mean)

cm_high_dp = cm[cm.dp > dp_mean]
print(cm_high_dp.dp.min())

Note that when selecting a subset of the data, a shallow copy (view) of the crystal map is obtained. This means that whatever changes made to `cm_high_dp` also change `cm`. When we want a deep copy, we use the `CrystalMap.deepcopy()` method

In [ ]:
cm_nobody_owns_me = cm[cm.dp > dp_mean].deepcopy()

We can chain the criteria

In [ ]:
cm[(cm.dp > 0.81) & (cm.phase_id == 1)]

# <a id='plot-maps'></a> 6. Plot maps

All map plotting is done via a so-called Matplotlib "projection" named "plot_map". To plot a phase map

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection="plot_map")
im = ax.plot_map(cm)

Hover over figure points to display the (x,y) position and orientations in that point!

Note that `plot_map()` wraps `matplotlib.axes.Axes.imshow`. All key word arguments in `plot_map()` are passed to `imshow()`, so be sure to check [its documentation]((https://matplotlib.org/api/_as_gen/matplotlib.axes.Axes.imshow.html?highlight=imshow#matplotlib.axes.Axes.imshow)) out for any additional arguments.

We can add any overlay, from any property with a value in each map point, to the map

In [ ]:
ax.add_overlay(cm, cm.dp)

To save our phase map with the scalebar and legend, but without white padding

In [ ]:
ax.remove_padding()
fig.savefig(
    datadir + 'phase_map.png',
    bbox_inches="tight",
    pad_inches=0,
)

To save phase map without scalebar, legend and white padding, and one image pixel per map point

In [ ]:
plt.imsave(
    datadir + 'phase_no_fluff.png',
    arr=im.get_array()  # 2D NumPy array, possibly with an RGB tuple in each element
)

We can plot any property with a value in each map point

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection="plot_map")
im = ax.plot_map(cm, cm.dp, cmap="inferno")

And change the colormap later if we want to

In [ ]:
im.set_cmap("viridis")

And add a colorbar if we want

In [ ]:
cbar = ax.add_colorbar(label="Dottproduct")

Which we can update if we mispelled the label or want other adjustements

In [ ]:
cbar.ax.set_ylabel("Dot product", rotation=270);

We can also plot orientation related values, like axis and angles etc., and restrict the color bar maximum

In [ ]:
# Get rotation angles in degrees
angles = cm.rotations.angle.data * 180 / np.pi

fig = plt.figure()
ax = fig.add_subplot(projection="plot_map")
im = ax.plot_map(cm, angles, vmax=angles.max() - 10)

ax.add_overlay(cm, cm.iq)

ax.add_colorbar(label="Rotation angle");

To plot only one phase, while passing custom:
* scalebar properties (https://matplotlib.org/mpl_toolkits/axes_grid/api/anchored_artists_api.html#mpl_toolkits.axes_grid1.anchored_artists.AnchoredSizeBar)
* legend properties (https://matplotlib.org/3.3.0/api/_as_gen/matplotlib.pyplot.legend.html)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection="plot_map")
im = ax.plot_map(
    cm["austenite"],
    scalebar=True,  # False for removed
    scalebar_properties={
        "loc": 4,  # 1: upper right, 2: upper left, etc. counter-clockwise
        "frameon": False,
        "sep": 6,  # Vertical spacing between bar and text
        "size_vertical": 0.2,  # Bar height
    },
    legend_properties={
        "framealpha": 1,  # 0: fully transparent, 1: opaque
        "handlelength": 1.5,  # Colored square width
        "handletextpad": 0.1,  # Horizontal space between square and text
        "borderpad": 0.1,
    },
)

Plot only a rectangle of the map

In [ ]:
cm2

In [ ]:
cm2 = cm[20:50, 40:90]

fig = plt.figure()
ax = fig.add_subplot(projection="plot_map")
ax.plot_map(cm2)
ax.add_overlay(cm2, cm2.dp)

Plot only parts of a map based on chained conditionals, like belonging to one phase or having a property value above a threshold

In [ ]:
# Conditional slicing
cm2 = cm[cm.dp > 0.81]

fig = plt.figure()
ax = fig.add_subplot(projection="plot_map")
ax.plot_map(cm2, cm2.iq, cmap="gray")
ax.add_colorbar("Image quality")

# Chained conditional slicing
cm2 = cm[(cm.dp > 0.81) & (cm.phase_id == 1)]

fig = plt.figure()
ax = fig.add_subplot(projection="plot_map")
ax.plot_map(cm2, cm2.dp, cmap="cividis")
ax.add_colorbar("Dot product");

Plot histogram of a property per phase

In [ ]:
# Property of interest
this_prop = 'dp'

# Plot phase map again to see color changes
fig = plt.figure()
ax = fig.add_subplot(projection="plot_map")
ax.plot_map(cm)

# Add overlay, passing str (can also pass numpy.ndarray)
ax.add_overlay(cm, this_prop)

# Remove figure padding
ax.remove_padding()

# Declare lists for plotting
data = []
labels = []
colors = []

# Get property values, name and color per phase
for _, p in cm.phases_in_data:
    labels.append(p.name)
    colors.append(p.color)

    # Accessing the property dictionary directly
    data.append(cm[p.name].prop[this_prop])
    # or indirectly
    #data.append(cm[p.name].dp)

# Nice bar plot with property histogram per phase
fig, ax = plt.subplots()
ax.hist(
    data,
    bins=20,
    histtype='bar',
    density=True,
    label=labels,
    color=colors
)
ax.set_xlabel(this_prop)
ax.set_ylabel("Frequency")
ax.legend();

Add a new property to the map, modify it, and plot it

In [ ]:
cm.prop["grain_boundary"] = 0

In [ ]:
cm[cm.dp < 0.81].grain_boundary = 1

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection="plot_map")
im = ax.plot_map(cm, cm.grain_boundary, cmap="gray")